In [1]:
import numpy as np 
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Cargamos los cvs desde google drive de train, también mas adelante el de test y gender submission.

In [3]:
data_path='/content/drive/MyDrive/train.csv'
df_train = pd.read_csv(data_path)
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_train['Cabin'].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

Vemos que tipos de datos tenemos en nuestro dataset, si hay variables numericas y categoricas. 

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Quitamos las conlumas que consideramos no nos puedan servir para el pronóstico.

In [6]:
train = df_train.drop(['Ticket', 'Cabin', 'Name'], axis = 1)
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

Realizo un one hoy encoding para la variable categoria sex, donde está másculina y femenino pasan a 0 y 1.
Lo mismo con la variable embarked, en el caso de age realizamos una imputación por el promedio de todos los datos. 

In [8]:
ohe = pd.get_dummies(train['Sex'], drop_first=True)
train.drop('Sex', axis=1, inplace=True)
train = train.join(ohe)
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,male
0,1,0,3,22.0,1,0,7.2500,S,1
1,2,1,1,38.0,1,0,71.2833,C,0
2,3,1,3,26.0,0,0,7.9250,S,0
3,4,1,1,35.0,1,0,53.1000,S,0
4,5,0,3,35.0,0,0,8.0500,S,1


In [9]:
ohe = pd.get_dummies(train['Embarked'], drop_first=True)
train.drop('Embarked', axis=1, inplace=True)
train = train.join(ohe)
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,1,0,3,22.0,1,0,7.2500,1,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,0,1


In [10]:
meanValue = np.round(train['Age'].mean(), 0)

In [11]:
train['Age'] = train['Age'].fillna(meanValue)
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,1,0,3,22.0,1,0,7.2500,1,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,0,1


In [12]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
male           0
Q              0
S              0
dtype: int64

In [13]:
train.isnull().values.any()

False

Seleccionamos de nuestro dataset de train la variable Y a predecir que es cuantos sobrevivieron al titanic. 

In [14]:
X = train.iloc[:, train.columns != 'Survived']
y = train.loc[:, 'Survived']

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [16]:
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)

In [17]:
#partición de data para entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.469135802, shuffle=True)

### Keras

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [19]:
inputs = X_train.shape[1]

10 neuronas con relu fue la que obtenemos mejores resultados.

Segunda capa oculta con 5 neuronas.

Tercera capa oculta con 2 neronas. 

In [45]:
#base secuencial para definir la estructura de la Red (SIEMPRE INICIAR CON SQUENTIAL)
clasificador = Sequential()

#primera capa oculta solo la primera necesita (input_dim = inputs
clasificador.add(Dense(input_dim = inputs, units=10, activation='relu', kernel_initializer='uniform'))

#segunda capa oculta ya en la segunda ya no se nesecita los input 
clasificador.add(Dense(units=5, activation='relu', kernel_initializer='uniform'))

#tercera capa oculta ya en la segunda ya no se nesecita los input
clasificador.add(Dense(units=2, activation='relu', kernel_initializer='uniform'))

#capa de salida
clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

Parametros de optimización el mejor es adam, podriamos haber usado el mini batch pero no me corrio por alguna razón.

In [46]:
#parametros de optimización
clasificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#parametros de entrenamiento.
clasificador.fit(X_train, y_train, batch_size=10, epochs=200)

Epoch 1/200
48/48 [==============================] - 1s 2ms/step - loss: 0.6918 - accuracy: 0.5920
Epoch 2/200
48/48 [==============================] - 0s 2ms/step - loss: 0.6886 - accuracy: 0.6089
Epoch 3/200
48/48 [==============================] - 0s 2ms/step - loss: 0.6817 - accuracy: 0.6448
Epoch 4/200
48/48 [==============================] - 0s 2ms/step - loss: 0.6583 - accuracy: 0.7653
Epoch 5/200
48/48 [==============================] - 0s 3ms/step - loss: 0.6188 - accuracy: 0.7759
Epoch 6/200
48/48 [==============================] - 0s 2ms/step - loss: 0.5917 - accuracy: 0.7844
Epoch 7/200
48/48 [==============================] - 0s 2ms/step - loss: 0.5750 - accuracy: 0.7928
Epoch 8/200
48/48 [==============================] - 0s 2ms/step - loss: 0.5621 - accuracy: 0.8055
Epoch 9/200
48/48 [==============================] - 0s 2ms/step - loss: 0.5522 - accuracy: 0.8055
Epoch 10/200
48/48 [==============================] - 0s 2ms/step - loss: 0.5419 - accuracy: 0.8076
Epoch 11/

In [47]:
path = '/content/drive/MyDrive/ANDRES' 
clasificador.save(path + 'carga1.h5')

In [48]:
# Cargamos la red.
new_model = keras.models.load_model(path + 'carga1.h5')

## Predicciones

In [49]:
data_path1='/content/drive/MyDrive/test.csv'
df_test = pd.read_csv(data_path1)
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [50]:
test = df_test.drop(['Ticket', 'Cabin', 'Name'], axis = 1)
test.head()
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,NaN,0,0,8.0500,S


In [26]:
ohe = pd.get_dummies(test['Sex'], drop_first=True)
test.drop('Sex', axis=1, inplace=True)
test = test.join(ohe)
test.head()
test

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,male
0,892,3,34.5,0,0,7.8292,Q,1
1,893,3,47.0,1,0,7.0000,S,0
2,894,2,62.0,0,0,9.6875,Q,1
3,895,3,27.0,0,0,8.6625,S,1
4,896,3,22.0,1,1,12.2875,S,0
...,...,...,...,...,...,...,...,...
413,1305,3,NaN,0,0,8.0500,S,1
414,1306,1,39.0,0,0,108.9000,C,0
415,1307,3,38.5,0,0,7.2500,S,1
416,1308,3,NaN,0,0,8.0500,S,1


In [27]:
ohe = pd.get_dummies(test['Embarked'], drop_first=True)
test.drop('Embarked', axis=1, inplace=True)
test = test.join(ohe)
test.head()
test

,PassengerId,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,892,3,34.5,0,0,7.8292,1,1,0
1,893,3,47.0,1,0,7.0000,0,0,1
2,894,2,62.0,0,0,9.6875,1,1,0
3,895,3,27.0,0,0,8.6625,1,0,1
4,896,3,22.0,1,1,12.2875,0,0,1
...,...,...,...,...,...,...,...,...,...
413,1305,3,NaN,0,0,8.0500,1,0,1
414,1306,1,39.0,0,0,108.9000,0,0,0
415,1307,3,38.5,0,0,7.2500,1,0,1
416,1308,3,NaN,0,0,8.0500,1,0,1


In [51]:
meanValue = np.round(test['Age'].mean(), 0)

In [52]:
test['Age'] = test['Age'].fillna(meanValue)
test.head()
test


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,30.0,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,30.0,0,0,8.0500,S


In [53]:
y_preds = new_model.predict(X_test)
y_preds_final = np.round(y_preds)
y_preds_final.shape


(418, 1)

In [55]:
from sklearn.metrics import accuracy_score

In [56]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


El Accuracy score segun el libro de ML in datascience, keras, tensorflow etc... dice que es el porcentaje de casos clasificadores correctamente.

In [57]:
accuracy_score(y_test, y_preds_final)


0.7751196172248804

In [58]:
#KFolds
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [59]:
def red_v1():
  #base secuencial para definir la estructura de la Red (SIEMPRE INICIAR CON SQUENTIAL)
  clasificador = Sequential()

  #primera capa oculta solo la primera necesita (input_dim = inputs
  clasificador.add(Dense(input_dim = inputs, units=10, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta ya en la segunda ya no se nesecita los input 
  clasificador.add(Dense(units=5, activation='relu', kernel_initializer='uniform'))

  clasificador.add(Dense(units=2, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))


  #parametros de optimización
  clasificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

In [60]:
clasificador = KerasClassifier(build_fn=red_v1, batch_size=25, nb_epoch=1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [61]:
kfolds = KFold(n_splits=10, shuffle=True)

accs = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.6s finished


In [62]:
accs

array([0.54166669, 0.60416669, 0.625     , 0.70212764, 0.63829786,
       0.65957445, 0.53191489, 0.65957445, 0.57446808, 0.55319148])

In [63]:
accs.mean()

0.6089982211589813

In [64]:
sub = pd.read_csv('/content/drive/MyDrive/gender_submission.csv')

In [65]:
sub['Survived'] = y_preds_final
sub.to_csv('submission_lightgbm.csv', index=False)

sub.head()

,PassengerId,Survived
0,892,0.0
1,893,1.0
2,894,0.0
3,895,0.0
4,896,0.0


In [66]:
from google.colab import files
sub.to_csv('carga3.csv') 
files.download('carga3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>